# Multiple answer cleanup
Attempt at dealing with fields that contain comma-separated lists of answers from questions with multiple-select answer options that also allow respondents to enter their own 'other' answer.

This notebook assumes the file named `responses_scrubbed_tagged.csv` has already been created by running the notebook named `free_text_tagging.ipynb`.  This notebook then creates a new data file called `responses_complete.csv` which has the complete set of final data.

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown, Latex

## Load tagged data set
This notebook assumes you have already scrubbed and tagged the data using the `free_text_tagging.ipynb` notebook first.

In [2]:
df = pd.read_csv('responses_scrubbed_tagged.csv', index_col=None, parse_dates=['date'])
df.sample(n=10) #random sample of rows

,date,neighborhood,demographic,child_bus_freq,child_bike_freq,child_driven_freq,child_drive_freq,child_walk_freq,walk_freq,bike_freq,...,no_bike_reason_2,no_bike_reason_3,drive_reason_0,drive_reason_1,drive_reason_2,drive_reason_3,drive_reason_4,drive_reason_5,problem_tags,suggestion_tags
549,2019-07-28 21:22:15,Old Post Road North,Adult without school kid,NaN,NaN,NaN,NaN,NaN,Once every few days,Rarely or never,...,I'm up several hills and a very steep driveway,NaN,see above,NaN,NaN,NaN,NaN,NaN,road surface,bike lanes
285,2019-07-04 12:52:18,Old Post Road North,Adult without school kid,NaN,NaN,NaN,NaN,NaN,"Every day, or almost every day",Once every few days,...,NaN,NaN,No - I do not drive,or prefer not to drive,Carrying Capacity of a car is better for weekl...,NaN,NaN,NaN,"n riverside,missing crosswalks,bank st,missing...","maintain sidewalks,more crosswalks,more street..."
269,2019-07-04 10:56:45,Old Post Road North,Adult with school kid,"Every day, or almost every day",Rarely or never - my children are too young,Rarely or never,Rarely or never - my children are too young,Rarely or never - my children are too young,"Every day, or almost every day",Rarely or never,...,NaN,NaN,Only drive to grocery because of bags.,NaN,NaN,NaN,NaN,NaN,"speeding,aggressive driving,cyclists breaking ...",educate cyclists
25,2019-07-03 15:39:56,Upper Village,Adult with school kid,"Every day, or almost every day",Rarely or never - my children are too young,Rarely or never,Rarely or never - my children are too young,Rarely or never - my children are too young,"Every day, or almost every day",Rarely or never,...,NaN,NaN,No - I do not drive,or prefer not to drive,NaN,NaN,NaN,NaN,"bushes on sidewalk, missing sidewalks, visibil...",more sidewalks
271,2019-07-04 11:02:25,Mount Airy,Senior,NaN,NaN,NaN,NaN,NaN,NaN,Once every few days,...,NaN,NaN,Distance from my airy,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,2019-07-03 19:00:11,Harmon,Senior,NaN,NaN,NaN,NaN,NaN,"Every day, or almost every day",Rarely or never,...,NaN,NaN,My own personal preference,NaN,NaN,NaN,NaN,NaN,"gottwald circle,cleveland,road surface,s river...","parking on one side only,reduce on-street park..."
28,2019-07-03 15:41:19,Old Post Road North,NaN,NaN,NaN,NaN,NaN,NaN,Rarely or never,Rarely or never,...,NaN,NaN,Safety concerns with walking,Safety concerns with bicycling,NaN,NaN,NaN,NaN,"speeding, old post n, high st",NaN
360,2019-07-06 15:43:13,Old Post Road North,Adult without school kid,NaN,NaN,NaN,NaN,NaN,Once in a while,Rarely or never,...,NaN,NaN,My own personal preference,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,2019-07-03 17:07:39,Half Moon Bay,Senior,NaN,NaN,NaN,NaN,NaN,"Every day, or almost every day",Once every few days,...,NaN,NaN,Sometimes I walk,sometimes I bike,sometimes I drive. Depends on the errand,distance and my time.,NaN,NaN,"municipal place,van wyck,missing sidewalks,sid...",NaN
364,2019-07-06 21:11:57,Mount Airy,Adult without school kid,NaN,NaN,NaN,NaN,NaN,"Every day, or almost every day",Once in a while,...,NaN,NaN,Safety concerns with bicycling,NaN,NaN,NaN,NaN,NaN,"mt airy,missing sidewalks,aggressive driving",NaN


## Answer options to each question
A dictionary containing lists of all the questions and the answer options that the user was asked to select from for each question in the Google Form.

In [3]:
# a dictionary of columns that contain comma-separated multiple answer options
multiple_answer_options = {
    'schools': ['CET', 'PVC', 'CHHS', 'Homeschooled'],
    'child_drive_reason': ['No - we do not drive, or prefer not to drive', 'Our own personal preference', 'Lack of available busing where we live', 'The bus schedule does not match our schedule', 'Safety concerns with buses', 'Safety concerns with walking', 'Safety concerns with bicycling', "My child's health condition"],
    'child_no_walk_reason': ['No - they walk a lot', 'My child does not like to walk', 'We live too far to walk', "We don't have time to walk", 'Fear of dangerous driving', 'Lack of adequate sidewalks', 'Lack of adequate crosswalks at busy intersections', 'Lack of crossing guards at busy intersections', "My child's health condition", 'Visually unappealing route'],
    'child_no_bike_reason': ['No - they bicycle a lot', 'My child does not like to bicycle', 'My child is too young to bicycle', 'We live too far to bicycle', 'Our own personal preference', 'Fear of dangerous driving', 'Lack of adequate bike lanes', "My child's health condition", 'Visually unappealing route', 'Hills'],
    'no_walk_reason': ['No - I walk a lot', 'I live too far to walk', 'I do not like to walk', "I don't have time to walk", 'Fear of dangerous driving', 'Lack of adequate sidewalks', 'Lack of adequate crosswalks at intersections', 'Lack of crossing guards at intersections', "My health condition", 'My own personal preference', 'Visually unappealing'], 
    'no_bike_reason': ['No - I bicycle a lot', 'I live too far to bicycle', 'I do not like to bicycle', "I don't have time to bicycle", 'Fear of dangerous driving', 'Lack of adequate bike lanes', 'My health condition', 'My own personal preference', 'Visually unappealing'],
    'drive_reason': ['No - I do not drive, or prefer not to drive', "I don't have time to walk or bicycle", 'Safety concerns with walking', 'Safety concerns with bicycling', 'My own health condition', 'My own personal preference'],
    'commuter_distance': ['Within Croton', 'Within Westchester', 'To/from New York City'],
}

# remember that there was also an 'other' field where users could enter custom answers... we'll deal with that later.

## Fix the mistake we made splitting these columns
In the `cleanup.ipynb` notebook, we split up columns with multiple answer options into multiple columns, such as `child_drive_reason_0`, `child_drive_reason1`, etc.  Merge them back together here.

In [4]:
# loop through each question title
for q in multiple_answer_options.keys():
    column_list = []
    for i in range(10):
        column_name = q + '_' + str(i) #auto-generate numbered column name
        if column_name in df.columns:
            column_list.append(column_name)
    if len(column_list) > 0:
        # create a new column with all answers comma-separated
        df[q] = df[column_list].apply(lambda x: ', '.join(x.fillna('').map(str)), axis=1)
        df[q] = df[q].str.strip(', ')
        
        # delete the old numbered columns
        df = df.drop(column_list, axis=1)        
df.columns

Index(['date', 'neighborhood', 'demographic', 'child_bus_freq',
       'child_bike_freq', 'child_driven_freq', 'child_drive_freq',
       'child_walk_freq', 'walk_freq', 'bike_freq', 'bikes_on_sidewalk',
       'self_jog_frequency', 'commutes', 'child_self_school',
       'child_self_bus_freq', 'child_self_bike_freq', 'child_self_driven_freq',
       'child_self_drive_freq', 'child_self_walk_freq', 'child_self_commutes',
       'child_self_has_children', 'commuter_distance', 'commuter_type',
       'commuter_walk_to_station_freq', 'commuter_bike_to_station_freq',
       'commuter_drive_to_station_freq', 'commuter_carpool_to_station_freq',
       'commuter_driven_to_station_freq', 'commuter_bus_to_station_freq',
       'feelings', 'problem_areas', 'drivers_are_safe', 'bicyclists_are_safe',
       'suggested_improvements', 'additional_comments', 'owns_business',
       'business_type', 'business_space', 'business_pedestrian_synergy',
       'business_bicyclists_synergy', 'business_bike_r

## Add in the tagging data
Tagging is done by the data analyst, and is assumed to be correct.  From here on, for simplicity, it will from here be stored and analyzed the same as if tags had been a question with multiple answer options with the most popular tags being treated as the answer options.

In [5]:
#add the most popular complaint tags to the dictionary of questions and their answer options
multiple_answer_options['problem_tags'] = [ 'speeding', 'missing sidewalks', 'cpa', 'sidewalk condition', 'aggressive driving', 'driver awareness', 's riverside', '129', 'maple', 'cyclists breaking rules', 'cleveland', 'road surface', 'municipal place', 'missing crosswalks', 'bushes on sidewalk', 'grand', 'narrow roads', 'mt airy', 'shoprite', 'benedict', 'five corners', 'van wyck', 'dummy light', 'crossing guard', 'gottwald circle', 'n riverside', 'poor lighting', 'croton commons', 'truesdale', 'hmb trail', 'cet', 'on-street parking', 'old post n', 'olcott', 'old post s', 'morningside' ]

#add the most popular suggestion tags to the dictionary of questions and their answer options
multiple_answer_options['suggestion_tags'] = [ 'bike lanes', 'more sidewalks', 'enforce speed', 'maintain sidewalks', 'reduce speed', 'educate cyclists', 'more crosswalks', 'speed bumps', 'no bike lanes', 'maintain crosswalks', 'enforce crosswalks', 'sidewalk on mt airy s', 'reduce on-street parking', 'more traffic lights', 'better lighting', 'more stop signs', 'dedicated pedestrian walk signals', 'more signs', 'improve route 9 access', 'educate pedestrians', 'outlaw cycling', 'enforce trimming of bushes', 'speed cameras', 'parking on one side only', 'bike racks', 'traffic calming', 'speed indicator signs' ]


## Limit to just the multiple select columns
For debugging

In [6]:
column_list = list(multiple_answer_options.keys())
df = df[column_list]
df = df.fillna('')
df.sample(n=10) #random sample of rows

,schools,child_drive_reason,child_no_walk_reason,child_no_bike_reason,no_walk_reason,no_bike_reason,drive_reason,commuter_distance,problem_tags,suggestion_tags
62,,,No - they walk a lot,My child is too young to bicycle,"I live too far to walk, I don't have time to walk",I do not like to bicycle,"I don't have time to walk or bicycle, Distance...",To/from New York City,poor lighting,better lighting
113,Briarcliff nursery school,"No - we do not drive, or prefer not to drive",Too young,Too young,"Lack of adequate sidewalks, Lack of adequate c...",Lack of adequate bike lanes,My own personal preference,,,
237,"CET, PVC",Our own personal preference,We don't have time to walk,My child does not like to bicycle,"I don't have time to walk, Lack of adequate si...",Lack of adequate bike lanes,"Safety concerns with bicycling, My own persona...",To/from New York City,"cpa,n riverside,station lot,missing crosswalks...","illuminated crosswalks,better lighting"
173,,,,,No - I walk a lot,,,,"cyclists breaking rules,driver awareness",enforce speed
522,,,,,My own personal preference,"Used to bike hurt back, will begin soon. 1-2x’...",My own personal preference,Within Westchester,129,educate cyclists
377,St. Augustine,Our own personal preference,We live too far to walk,We live too far to bicycle,No - I walk a lot,I don't have time to bicycle,My own personal preference,To/from New York City,cyclists breaking rules,
495,,,,,I live too far to walk,I don't have time to bicycle,I don't have time to walk or bicycle,To/from New York City,"cpa,road surface",
553,CET,The bus schedule does not match our schedule,Too young.,My child is too young to bicycle,I don't have time to walk,"Fear of dangerous driving, We live on Mt Airy ...",I don't have time to walk or bicycle,To/from New York City,"sidewalk condition, maple, bushes on sidewalk,...","enforce speed, maintain sidewalks, bike lanes"
215,,,,,My health condition,My health condition,My own health condition,Within Croton,,
371,CET,"No - we do not drive, or prefer not to drive",We live too far to walk,My child is too young to bicycle,Lack of adequate sidewalks,,Going somewhere too far to walk,,"missing sidewalks,sidewalk condition",


## Remove commas from any given answer option so we can later split by comma cleanly

In [7]:
# loop through all questions that allow more than one answer
for question, answer_list in multiple_answer_options.items():
    # loop through every answer in the answer list for this question
    for answer in answer_list:
        # remove the commas, from each answer, if any
        if ',' in answer:
            # update the answers with a cleaned version
            df[question] = df[question].str.replace(answer, answer.replace(',', '') )
            
    # remove commas from the list of answers in the question/answer dictionary
    multiple_answer_options[question] = [answer.replace(',' , '') for answer in answer_list]

# show the cleaned up answer options
#multiple_answer_options

## Tally answers
Indicate which users selected which answer options, and put any freeform text answers that users entered by clicking the 'other' option in Google Forms into '_other' columns.

In [8]:
def categorize_answers(response, question, answer_list):    
    """
    Takes a given set of responses to a multi-select question, and puts them into a well-formated Series of dummy values.
    
    @param response A list of the user's responses to this question
    @param question The title of the question (these are codes)
    @param answer_list A list of the answer options that were presented to the user in the Google Form
    """
    
    # set up some lists that we will use to collect responses as dummy values
    indices = [] # will contain the text of the answer option that the user selected
    values = [] # will contain dummy values (1s) for each answer option the user selected, NaNs otherwise.
    
    # skip any nan or blank values
    if str(response) != 'nan' and type(answer_list) != float and type(question) != float and type(response) != float:
        #display(question + " - " + str(response))
        for answer_option in response:
            answer_option = answer_option.strip() # remove any leading/trailing whitespace
            if answer_option in answer_list and question + '_' + answer_option not in indices:
                indices.append(question + '_' + answer_option)
                values.append(1)
            elif answer_option.strip() != '' and question + '_' + 'other' not in indices:
                indices.append(question + '_' + 'other')
                values.append(answer_option)
    
        # put it all into a pandas Series
        response = pd.Series(values,index=indices).fillna(0).astype(str)
        
    return response

# slice up answers by comma and give each its own column
df2 = pd.DataFrame() # build a new dataframe from scratch

# loop through each question/answer item
for question, answer_list in multiple_answer_options.items():
    # get a new dataframe with the questions as indexes and the valuess as dummy values indicating user selections
    d = df[question].str.split(',\s*').apply(categorize_answers, question=question, answer_list=answer_list)
    df2 = df2.append(d) # append this as a row to the new dataframe

# show the new dataframe
df2.sample(n=10) #random sample of rows

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,child_drive_reason_Lack of available busing where we live,child_drive_reason_My child's health condition,child_drive_reason_No - we do not drive or prefer not to drive,child_drive_reason_Our own personal preference,child_drive_reason_Safety concerns with bicycling,child_drive_reason_Safety concerns with buses,child_drive_reason_Safety concerns with walking,child_drive_reason_The bus schedule does not match our schedule,child_drive_reason_other,child_no_bike_reason_Fear of dangerous driving,...,suggestion_tags_other,suggestion_tags_outlaw cycling,suggestion_tags_parking on one side only,suggestion_tags_reduce on-street parking,suggestion_tags_reduce speed,suggestion_tags_sidewalk on mt airy s,suggestion_tags_speed bumps,suggestion_tags_speed cameras,suggestion_tags_speed indicator signs,suggestion_tags_traffic calming
288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Convert values to ints or booleans, where possible
Most columns contain NaNs or 1s.  NaNs are converted to 0s then to booleans, if desired.  Integer dummy values are perhaps more useful so we can take counts of responses by adding the ints up.  The 'other' columns contain text and are left alone.

In [9]:
# loop through each column
for col in df2.columns:
    try:
        # try to convert to int
        df2[col] = df2[col].fillna(0).astype(int)#.astype(bool)
    except:
        # if this column contains text, skip...
        continue

## Group rows by 'id' so there is one row for each respondent

In [10]:
# add up all values in each column to get a full set of values or each id
# this loses the 'other' columns, since they are not numeric
df3 = df2.groupby(df2.index).sum()

# show the new grouped data... note that it is missing the 'other' fields... more on that later
df3.sample(n=10) #random sample of rows

,child_drive_reason_Lack of available busing where we live,child_drive_reason_My child's health condition,child_drive_reason_No - we do not drive or prefer not to drive,child_drive_reason_Our own personal preference,child_drive_reason_Safety concerns with bicycling,child_drive_reason_Safety concerns with buses,child_drive_reason_Safety concerns with walking,child_drive_reason_The bus schedule does not match our schedule,child_no_bike_reason_Fear of dangerous driving,child_no_bike_reason_Hills,...,suggestion_tags_no bike lanes,suggestion_tags_outlaw cycling,suggestion_tags_parking on one side only,suggestion_tags_reduce on-street parking,suggestion_tags_reduce speed,suggestion_tags_sidewalk on mt airy s,suggestion_tags_speed bumps,suggestion_tags_speed cameras,suggestion_tags_speed indicator signs,suggestion_tags_traffic calming
68,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
118,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
484,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
549,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
296,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
404,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
428,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
533,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
306,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Combine the selected answer options with the 'other' responses users gave

In [11]:
# loop through all questions
for question, answer_options in multiple_answer_options.items():
    # get a dataframe of just the 'other' answers to this question
    other_df = df2[pd.notnull(df2[question + '_other'])][question + '_other'].to_frame()
    # concatenate those responses to the original dataframe
    df3 = df3.join(other_df)
    
# show the data
df3.sample(n=10) #random sample of rows

,child_drive_reason_Lack of available busing where we live,child_drive_reason_My child's health condition,child_drive_reason_No - we do not drive or prefer not to drive,child_drive_reason_Our own personal preference,child_drive_reason_Safety concerns with bicycling,child_drive_reason_Safety concerns with buses,child_drive_reason_Safety concerns with walking,child_drive_reason_The bus schedule does not match our schedule,child_no_bike_reason_Fear of dangerous driving,child_no_bike_reason_Hills,...,schools_other,child_drive_reason_other,child_no_walk_reason_other,child_no_bike_reason_other,no_walk_reason_other,no_bike_reason_other,drive_reason_other,commuter_distance_other,problem_tags_other,suggestion_tags_other
187,0,0,1,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,albany post road,NaN
8,0,0,0,0,0,0,0,1,0,0,...,NaN,NaN,Dicey route,NaN,NaN,NaN,Time saving to drive,NaN,NaN,NaN
61,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185,0,0,0,0,0,0,0,0,0,0,...,NaN,Teenagers can't wake up in morning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
391,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dutchess County,NaN,NaN
488,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,I drop my kids off at Happy Hearts and can't b...,NaN,NaN,NaN
234,0,0,0,0,0,0,0,0,0,1,...,NaN,NaN,On the far side,NaN,NaN,NaN,Hills,Both nyc and westChester,NaN,NaN
455,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,The roads are not made for cycling.,NaN,NaN,NaN,NaN
556,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,I currently don’t own a bicycle,NaN,NaN,NaN,NaN


## Order the columns alphabetically
So it looks a bit more orderly

In [12]:
column_names = df3.columns.sort_values()
df3 = df3[column_names]
df3.sample(n=10) #random sample of rows

,child_drive_reason_Lack of available busing where we live,child_drive_reason_My child's health condition,child_drive_reason_No - we do not drive or prefer not to drive,child_drive_reason_Our own personal preference,child_drive_reason_Safety concerns with bicycling,child_drive_reason_Safety concerns with buses,child_drive_reason_Safety concerns with walking,child_drive_reason_The bus schedule does not match our schedule,child_drive_reason_other,child_no_bike_reason_Fear of dangerous driving,...,suggestion_tags_other,suggestion_tags_outlaw cycling,suggestion_tags_parking on one side only,suggestion_tags_reduce on-street parking,suggestion_tags_reduce speed,suggestion_tags_sidewalk on mt airy s,suggestion_tags_speed bumps,suggestion_tags_speed cameras,suggestion_tags_speed indicator signs,suggestion_tags_traffic calming
159,0,0,0,0,1,1,1,0,NaN,1,...,NaN,0,0,0,0,0,0,0,0,0
101,0,0,0,1,0,0,0,0,NaN,0,...,improve street sweeping,0,0,0,0,0,0,0,0,0
396,0,0,0,0,0,0,0,0,NaN,0,...,NaN,0,0,0,0,0,0,0,0,0
62,0,0,0,0,0,0,0,0,NaN,0,...,NaN,0,0,0,0,0,0,0,0,0
175,0,0,0,0,0,0,0,0,NaN,0,...,NaN,0,0,0,0,0,0,0,0,0
384,0,0,0,0,0,0,0,0,NaN,0,...,NaN,0,0,0,0,0,1,0,0,0
29,0,0,0,0,0,0,0,1,NaN,1,...,NaN,0,0,0,1,1,0,0,0,0
437,0,0,0,0,0,0,0,0,NaN,0,...,NaN,0,0,1,0,0,0,0,0,0
535,0,1,0,0,0,0,0,0,NaN,0,...,NaN,0,0,0,0,0,0,0,0,0
230,0,0,1,0,0,0,0,0,NaN,0,...,NaN,0,0,0,0,0,0,0,0,0


## Merge with remaining data

In [13]:
# load up the original data from scratch again
df = pd.read_csv('responses_scrubbed_tagged.csv', index_col=None, parse_dates=['date'])

# merge the new columns with the old dataframe based on the indexes of each row
df_complete = pd.merge(df, df3, right_index=True, left_index=True)

df_complete.sample(n=10) #random sample of rows

,date,neighborhood,demographic,child_bus_freq,child_bike_freq,child_driven_freq,child_drive_freq,child_walk_freq,walk_freq,bike_freq,...,suggestion_tags_other,suggestion_tags_outlaw cycling,suggestion_tags_parking on one side only,suggestion_tags_reduce on-street parking,suggestion_tags_reduce speed,suggestion_tags_sidewalk on mt airy s,suggestion_tags_speed bumps,suggestion_tags_speed cameras,suggestion_tags_speed indicator signs,suggestion_tags_traffic calming
7,2019-05-22 16:46:09,Outside Croton,Senior,NaN,NaN,NaN,NaN,NaN,Once in a while,Once in a while,...,sharrows,0,0,0,0,0,0,0,0,0
188,2019-07-03 19:24:39,Outside Croton,Adult without school kid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
249,2019-07-04 07:42:03,Old Post Road North,Adult without school kid,NaN,NaN,NaN,NaN,NaN,Rarely or never,Rarely or never,...,NaN,0,0,0,0,0,0,0,0,0
320,2019-07-04 22:37:35,Harmon,Senior,NaN,NaN,NaN,NaN,NaN,Once in a while,Rarely or never,...,NaN,0,0,0,0,0,0,0,0,0
361,2019-07-06 17:10:40,Harmon,Senior,NaN,NaN,NaN,NaN,NaN,Once in a while,Once in a while,...,NaN,0,0,0,1,0,0,0,0,0
365,2019-07-06 21:57:20,Harmon,Adult with school kid,"Every day, or almost every day",Rarely or never - my children are too young,Once every few days,Rarely or never - my children are too young,Rarely or never - my children are too young,Once every few days,Once in a while,...,buses to pvc within 1 mile,0,0,0,0,0,0,0,0,0
166,2019-07-03 18:18:06,Harmon,Adult with school kid,"Every day, or almost every day",Rarely or never - for other reasons,Rarely or never,Rarely or never - for other reasons,Rarely or never - for other reasons,"Every day, or almost every day","Every day, or almost every day",...,NaN,0,0,0,0,0,0,0,0,0
551,2019-07-29 13:37:55,Upper Village,Adult without school kid,NaN,NaN,NaN,NaN,NaN,Once in a while,Once in a while,...,NaN,0,0,0,0,0,0,0,0,0
339,2019-07-05 15:19:54,Old Post Road North,Senior,NaN,NaN,NaN,NaN,NaN,"Every day, or almost every day","Every day, or almost every day",...,NaN,0,0,0,0,0,0,0,0,0
435,2019-07-09 19:13:51,Harmon,Adult with school kid,Rarely or never,Rarely or never - for other reasons,Rarely or never,Rarely or never - my children are too young,"Every day, or almost every day","Every day, or almost every day",Rarely or never,...,stop signs at gottwald circle,0,0,0,0,0,0,0,0,0


## Group together some related sets of tags from the free text responses
There were several tags given for complaints about speeding, sidewalks, crosswalks, etc.... Make aggregate columns to make these things easier to search for.

In [14]:
def aggregate_tags(row, related_tags):
    """
    This method looks for a dummy value in a set of related fields.
    Returns 1 if found, 0 others.
    """
    # loop through each tag in the group
    for tag in related_tags:
        # if we find a one, then we're done
        if row[tag] == 1:
            return 1
    # otherwise, we did not find what we were looking for
    return 0

# make some new aggregate columns around topics that are otherwise represented across several fields

# a bunch of tags all related to concerns with drivers' behavior and speed
related_tags = ['problem_tags_driver awareness', 'problem_tags_aggressive driving', 'problem_tags_speeding', 'suggestion_tags_reduce speed', 'suggestion_tags_enforce speed', 'suggestion_tags_speed bumps', 'suggestion_tags_speed cameras', 'suggestion_tags_speed indicator signs', 'suggestion_tags_traffic calming']
df_complete['concerns_with_drivers'] = df_complete.apply (lambda row: aggregate_tags(row, related_tags), axis=1)

# a bunch of tags all related to concerns with sidewalks
related_tags = ['problem_tags_missing sidewalks', 'problem_tags_sidewalk condition', 'suggestion_tags_more sidewalks', 'suggestion_tags_maintain sidewalks']
df_complete['concerns_with_sidewalks'] = df_complete.apply (lambda row: aggregate_tags(row, related_tags), axis=1)

# a bunch of tags all related to concerns with crosswalks
related_tags = ['problem_tags_missing crosswalks', 'suggestion_tags_more crosswalks', 'suggestion_tags_maintain crosswalks', 'suggestion_tags_enforce crosswalks']
df_complete['concerns_with_crosswalks'] = df_complete.apply (lambda row: aggregate_tags(row, related_tags), axis=1)


## Save to new CSV file

In [15]:
df_complete.to_csv('responses_complete.csv', index=False)

## Done!